# Pivot Tables with Python and Pandas

Next, we jump into creating pivot tables using Python and Pandas. First, we will have to do a bit of data cleaning though.

Import data and have a look at the first couple lines of data.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("nhex.csv", dtype={'Year':str, 'Current dollars \nper capita':str}, na_values=[' --'])

print(data.head())

print(data.columns.tolist()) ## generate a list to make the next step easier

   Year               Jurisdiction                             Sector  \
0  1996  Newfoundland and Labrador  Provincial/territorial government   
1  1996  Newfoundland and Labrador  Provincial/territorial government   
2  1996  Newfoundland and Labrador  Provincial/territorial government   
3  1996  Newfoundland and Labrador  Provincial/territorial government   
4  1996  Newfoundland and Labrador  Provincial/territorial government   

  Use of funds Age group Gender Current dollars Current dollars \nper capita  \
0        Total        <1      F               —                            —   
1        Total       1–4      F               —                            —   
2        Total       5–9      F               —                            —   
3        Total     10–14      F               —                            —   
4        Total     15–19      F               —                            —   

   Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13  \

Now let's select just the columns of interest. I omitted 'Sector' becuase I already know all values are the same (i.e. government) and  'Current dollars' because I want to know the per capita amount. 

Also, I am only interested in looking at the data for the most recent year in the dataset, 2014, so we'll filter that out.

In [2]:
data = data[['Year','Jurisdiction', 'Sector', 'Use of funds', 'Age group', 'Gender', 'Current dollars \nper capita']]
data.rename(columns={'Current dollars \nper capita': 'Dollars per capita'}, inplace=True )

data = data.loc[data['Year'] == '2014']

data.info() ## check that there are no Null values and that the column renaming worked 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3360 entries, 60480 to 63839
Data columns (total 7 columns):
Year                  3360 non-null object
Jurisdiction          3360 non-null object
Sector                3360 non-null object
Use of funds          3360 non-null object
Age group             3360 non-null object
Gender                3360 non-null object
Dollars per capita    3360 non-null object
dtypes: object(7)
memory usage: 210.0+ KB


In [3]:
data['Dollars per capita'] = data['Dollars per capita'].astype(str)

data['Dollars per capita'] = data['Dollars per capita'].apply(lambda x: pd.to_numeric(x.replace(',',''), errors='coerce'))

data['Dollars per capita'].head() 

60480    10762.0
60481     1708.0
60482     1642.0
60483     1568.0
60484     2187.0
Name: Dollars per capita, dtype: float64

Let's fix the values in the Age Group column. Currently, there are a mix of datatypes that include ranges with dashes, comparator signs and addition signs. The issue with this is the pivot table columns will get sorted into the wrong order. 

To solve this, we can change the Age Group values to the respective lower bound (e.g. <1 --> 0, 1-4 --> 1, and 90+ --> 90). We can then rename the columns after the pivot table is created. 

In [4]:
import re
print(data['Age group'][0:5])

## grab Age Group Column
## apply funciton that removes extra characters
def fixAgeGroup(val):
    if val == '<1':
        return 0
    if val == '90+':
        return 90
    else:
        new_val = re.search(r"([^–]*)", val).group(1)
        return int(new_val)
    
data['Age group'] = data['Age group'].apply(fixAgeGroup)


print(data['Age group'][0:5])

60480       <1
60481      1–4
60482      5–9
60483    10–14
60484    15–19
Name: Age group, dtype: object
60480     0
60481     1
60482     5
60483    10
60484    15
Name: Age group, dtype: int64


Next, let's do some pivoting!

We'll use the [pandas.pivot_table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html) function. We need to specify each of hose four options that we disussed earlier:

1. **Filter** - I have already filtered for the year 2014 
2. **Column labels** - specified by the arugment `columns="Age group"`
3. **Row labels** - specified by the argument `index=['Jurisdiction', 'Use of funds']`
4. **Summation values** - specified by the argument `values='Dollars per capita'`

After creating the pivot table, we'll change the column names back to their original Age Groups. 


In [7]:
data_pivot = data.pivot_table(index=['Jurisdiction', 'Use of funds'], values='Dollars per capita', columns="Age group", aggfunc="sum")

data_pivot.columns = ['<1', '1-4', '5-9', '10-14,', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90+']

data_pivot

<1     1-4     5-9  10-14,  \
Jurisdiction         Use of funds                                           
Alberta              Drugs                   20.0    30.0    45.0    68.0   
                     Hospitals            24939.0  1059.0   568.0   828.0   
                     Other institutions       0.0     0.0     0.0    31.0   
                     Other professionals      6.0    98.0   291.0   320.0   
                     Physicians            3433.0  1147.0   870.0   852.0   
                     Total                29748.0  3685.0  3126.0  3450.0   
British Columbia     Drugs                   12.0    22.0    46.0    79.0   
                     Hospitals            17501.0   577.0   378.0   549.0   
                     Other institutions       3.0     4.0     3.0     2.0   
                     Other professionals      6.0    59.0   164.0   166.0   
                     Physicians            2654.0   702.0   557.0   526.0   
                     Total                21916.0  3103.0  2885.0  3062.0   
Canada               Drugs                   36.0    33.0    52.0    81.0   
                     Hospitals            17621.0   769.0   401.0   553.0   
                     Other institutions       2.0     3.0     2.0    12.0   
                     Other professionals      8.0    65.0   178.0   114.0   
                     Physicians            2511.0   911.0   633.0   621.0   
                     Total                21571.0  3171.0  2656.0  2769.0   
Manitoba             Drugs                    5.0    25.0    46.0    89.0   
                     Hospitals            16412.0   670.0   340.0   504.0   
                     Other institutions       7.0     9.0     5.0    45.0   
                     Other professionals     14.0    38.0    84.0    97.0   
                     Physicians            1554.0   895.0   585.0   649.0   
                     Total                19675.0  3322.0  2743.0  3067.0   
New Brunswick        Drugs                    7.0    18.0    35.0    51.0   
                     Hospitals            12909.0   510.0   270.0   441.0   
                     Other institutions       0.0     0.0     0.0     8.0   
                     Other professionals      NaN     NaN     NaN     NaN   
                     Physicians            2259.0   681.0   468.0  1366.0   
                     Total                16306.0  2369.0  2069.0  3161.0   
...                                           ...     ...     ...     ...   
Ontario              Drugs                   18.0    31.0    47.0    64.0   
                     Hospitals            17532.0   724.0   370.0   530.0   
                     Other institutions       2.0     3.0     1.0     4.0   
                     Other professionals     12.0    30.0    55.0    57.0   
                     Physicians            2510.0   832.0   585.0   550.0   
                     Total                21402.0  2951.0  2387.0  2535.0   
Prince Edward Island Drugs                    5.0    14.0    28.0    47.0   
                     Hospitals            14673.0   663.0   329.0   506.0   
                     Other institutions       0.0     0.0     0.0     0.0   
                     Other professionals      NaN     NaN     NaN     NaN   
                     Physicians            1797.0  1142.0  2108.0  4276.0   
                     Total                17676.0  3100.0  3903.0  6166.0   
Quebec               Drugs                  101.0    37.0    59.0   104.0   
                     Hospitals            13062.0   720.0   349.0   415.0   
                     Other institutions       3.0     5.0     2.0    13.0   
                     Other professionals      3.0   108.0   338.0    50.0   
                     Physicians            2114.0   974.0   614.0   548.0   
                     Total                16442.0  3000.0  2518.0  2286.0   
Saskatchewan         Drugs                   33.0    78.0   113.0   199.0   
                     Hospitals            1

All done! 

Since we have cleaned up the dataframe a bit, let's export it to a new csv for use with our SQL pivot table.

In [6]:
data.to_csv('nhex-fixed.csv')